# MPI in Jupyter


MPI [slides](https://docs.google.com/presentation/d/1JkGep7kRYJeB5SL-0z10n8GmFAyaAn3mDoIbB6pK6wQ/edit?usp=sharing)

We will use: 
 
 - [MPI4py](http://mpi4py.readthedocs.io/en/stable/)
 - [Ipython parallel](http://ipyparallel.readthedocs.io/) 

However, we will limit usage of `ipyparallel` features to minimum. Most of parallel logic will be in MPI standard.

## The classical way

 - Write a file and use mpirun directly.

In [28]:
%%writefile mpi001.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print "OK, rank= ",rank,size

Overwriting mpi001.py


In [19]:
!mpirun -n 4 /opt/conda/envs/py27/bin/python mpi001.py

OK, rank=  0 4
OK, rank=  1 4
OK, rank=  2 4
OK, rank=  3 4


### Problems:
 - wrong python
 - where is my file ?

In [20]:
!mpirun -n 4 python2 mpi001.py

Traceback (most recent call last):
  File "mpi001.py", line 2, in <module>
    from mpi4py import MPI
ImportError: No module named mpi4py
Traceback (most recent call last):
  File "mpi001.py", line 2, in <module>
    from mpi4py import MPI
ImportError: No module named mpi4py
Traceback (most recent call last):
  File "mpi001.py", line 2, in <module>
    from mpi4py import MPI
ImportError: No module named mpi4py
Traceback (most recent call last):
  File "mpi001.py", line 2, in <module>
    from mpi4py import MPI
ImportError: No module named mpi4py


In [21]:
!mpirun -n 4 python mpi001.py

  File "mpi001.py", line 7
    print "OK, rank= ",rank,size
                     ^
SyntaxError: Missing parentheses in call to 'print'
  File "mpi001.py", line 7
    print "OK, rank= ",rank,size
                     ^
SyntaxError: Missing parentheses in call to 'print'
  File "mpi001.py", line 7
    print "OK, rank= ",rank,size
                     ^
SyntaxError: Missing parentheses in call to 'print'
  File "mpi001.py", line 7
    print "OK, rank= ",rank,size
                     ^
SyntaxError: Missing parentheses in call to 'print'




## Use ipyparallel cluster:

   First - setup an MPI profile!


In [8]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')
print(c.ids)
view = c[:]
view.activate()

[0, 1, 2, 3]


### First Way:

 - write a file and do ```view.run("myfile.py")```


In [14]:
%%writefile mpi001.py

from mpi4py import MPI
import time
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
time.sleep(5)
import os
a = 1.23     
print ("OK, rank= ",rank,size,os.getpid())

Overwriting mpi001.py


In [15]:
ar = view.run('mpi001.py')

In [16]:
ar

<AsyncResult: execute>

In [21]:
ar.display_outputs()

[stdout:0] OK, rank=  3 4 21251
[stdout:1] OK, rank=  2 4 21250
[stdout:2] OK, rank=  1 4 21249
[stdout:3] OK, rank=  0 4 21248


In [22]:
ar = view.execute('mpi001.py')

In [23]:
view['a']

[1.23, 1.23, 1.23, 1.23]

this is equivalent to:


In [24]:
view.pull('a', block=True)

[1.23, 1.23, 1.23, 1.23]

### Second Way:

  - use magics: `%%px ....`  
     

** important!**

In [55]:
view.activate()

In [56]:
%%px 

from mpi4py import MPI
import time
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
time.sleep(5)
import os
a = 1.23
print ("OK, rank= ",rank,size,os.getpid())

<AsyncResult: execute>

In [57]:
%pxresult

[stdout:0] OK, rank=  2 4 7914
[stdout:1] OK, rank=  0 4 7912
[stdout:2] OK, rank=  3 4 7915
[stdout:3] OK, rank=  1 4 7913


### Worth trying:
    
  - `%%px --block`
  - `%%px --targets ::2`
  - `%autopx`

In [20]:
%%px --block --target 1:3:

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.rank
print(comm.size)

[stdout:1] 4
[stdout:2] 4
